<a href="https://colab.research.google.com/github/paryoja/deeplearningstudy/blob/master/Regression_(4)_Basis_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basis function

다음과 같은 데이터를 linear regression으로 fitting 할 수 있을 까요?


In [0]:
import numpy as np

dx = np.linspace(0, 2, 100)
dy = 5 * dx ** 3 + np.random.randn(100) / 10 # y = 5 * x^3

import matplotlib.pyplot as plt
%matplotlib inline 

plt.plot(dx, dy)
plt.show()

기존 linear regression을 수행하면 어떻게 될까요?

In [0]:
import tensorflow as tf

# 각 데이터는 1차원이니까 w도 1개만 필요합니다.
w = tf.Variable(tf.random_normal(1), dtype=tf.float32)
# bias 는 1개
b = tf.Variable(tf.zeros(1), dtype=tf.float32)

# 이번에는 placeholder를 이용해보겠습니다.
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)

# 예측값은 wx + b로 계산하고,
hypothesis = w * x + b

# loss는 정답값과 예측값의 차의 제곱의 합입니다.
loss = tf.reduce_mean(tf.square(hypothesis - y))

# gradient descent optimizer를 쓰고
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(10000):
  # optimize를 합니다.  
  _, l = sess.run([train_op, loss], feed_dict={x: dx, y: dy})
  
  if i % 1000 == 0:
    print('step', i, 'loss', l)
    

w_value, b_value = sess.run([w, b])
print('w', w_value)
print('b', b_value)

# 주어진 dx 값에 따른 예측값을 찍어봅니다
prediction = sess.run(hypothesis, feed_dict={x: dx, y: dy})

plt.plot(dx, dy, 'b')
plt.plot(dx, prediction, 'r')
plt.show()

예상대로 직선으로는 이러한 곡선을 제대로 표현하지 못합니다.


Linear regression은 정녕 직선밖에 맞출수 없는 운명인 것일까요?


어디선가 데이터의 요정이 나타납니다.




그리고 데이터를 다음과 같이 변경해 줍니다.

new_dx = np.pow(dx, 3)

그리고 말합니다.

"이 데이터로 linear regression을 해보거라"


In [0]:
new_dx = np.power(dx, 3)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(10000):
  
  _, l = sess.run([train_op, loss], feed_dict={x: new_dx, y: dy}) # new_dx 가 들어갑니다!
  
  if i % 1000 == 0:
    print('step', i, 'loss', l)
  
w_value, b_value = sess.run([w, b])


prediction = sess.run(hypothesis, feed_dict={x: new_dx, y: dy}) # new_dx 가 들어갑니다!

plt.title("바뀐 new_dx 기준으로 plot")
plt.plot(new_dx, dy, 'b')
plt.plot(new_dx, prediction, 'r')

plt.show()


plt.figure()
plt.title("원본 dx 기준으로 plot")
plt.plot(dx, dy, 'b')
plt.plot(dx, prediction, 'r')

plt.show()

Regression을 하는 입장에서 dx는 그냥 누군가가 던져준겁니다.

여기에 어떤 연산을 해서 주든간에 linear regression은 직선을 이용하여 맞춰줍니다.


데이터의 요정이 데이터를 바꿔줬을때, 결과값과 선형적인 관계를 이루고 있을 수도 있습니다.

특히 데이터가 어떠한 공식을 통해 바뀌었는지를 안다면,

그 공식을 이용하면 쉽게 fitting 할 수 있습니다.

다음을 fitting 해 보세요.

힌트는 x 값 하나를 \[x ** 3, np.sin(x * 100), np.exp(dx)\] 의 3차원 데이터로 만들고

이를 3차원 regression을 진행하면 됩니다.

In [0]:
import numpy as np

dx = np.linspace(0, 2, 100)
dy = 5 * dx ** 3 + 100 * np.sin(dx*100) - 8 * np.exp(dx) + np.random.randn(100) / 10 # y = x^3



import matplotlib.pyplot as plt
%matplotlib inline 

plt.plot(dx, dy)
plt.show()


이 처럼 데이터를 생성한 수식을 알고 있다면, linear regression은 쉽게 데이터를 fitting 할 수 있습니다.


위 수식에서 x^3, sin(100x), exp(x) 와 같은 데이터를 생성할때 쓰인 함수들을 basis function 이라고 부릅니다.

자 이제 두려울 것이 없습니다.

"데이터가 어떤 식으로 생성됬는지만 안다면" noise의 영향 정도만 틀리고 추세를 맞출 수 있습니다!


그럼 주식 데이터를 regression 해서 주가를 예측해 보도록 하죠!


"주가 데이터가 어떤 식으로 생성됬는지, 수학적 모델을 만드시면" 제가 linear regression을 해보겠습니다!

그런데 말입니다.


"주가의 수학적 모델이 어떻게 되죠?"


# 임의의 함수에 대한 basis function


주가에 영향을 주는 변수는 다양합니다.

환율, 유가, 국제 정세, 정책, 기업의 실적발표 등등  다양한 변수들이 복잡하게 얽혀있습니다.


환율을 x1, 유가를 x2, .... 모든 변수를 망라했을 때,

주식가격을 결정하는 함수 f(x1, x2, ...., ) 의 형태를 안다면, 이를 근사화하는 linear regression을 만들수 있겠지만,

저희는 f(x1, x2, ... ) 가 어떠한 함수로 이루어져 있는지, 그게 다항함수인지(x, x^2, x^3, ...), 삼각함수인지(sin, cos), 지수함수인지(exp)도 모릅니다.

그렇다면 어느 basis function을 골라 써야 하는데 

어느 basis function이 좋은지는 경험과 실험 결과에 의존하는 수밖에 없습니다.

Taylor 급수를 기억하시는 분이 계시다면

https://ko.wikipedia.org/wiki/%ED%85%8C%EC%9D%BC%EB%9F%AC_%EA%B8%89%EC%88%98

다항함수의 무한합을 통해서 모든 무한번 미분가능한 함수를 표현 할 수 있다는 사실을 알고 계실 텐데,

다항함수의 첫 3개 항을 이용하여 

다음과 같은 sin 함수를 fitting 해봅시다




In [0]:
import numpy as np

rng = np.random.RandomState(1)

dx = 2 * np.pi * rng.rand(100)
dy = 10 * np.sin(dx) + rng.randn(100) / 1  # y = 10 sin (x)



import matplotlib.pyplot as plt
%matplotlib inline 

plt.scatter(dx, dy)
plt.show()


기존의 데이터 마이닝 기법들은

데이터 사이언티스트가 적합하다고 판단한 basis function을 이용하여 데이터를 fitting 했습니다.


만약 어느 기술인가가 어느 basis를 쓰는 것이 좋더라 알려준다면 어떨까요?